In [3]:
# Cell 1: Imports
from PIL import Image
import os
from tqdm import tqdm

# Cell 2: Function Definition
# Cell 2: Function Definition (recursive)
def downsample_images(input_dir, output_dir, target_size=(128, 128)):
    """
    Downsamples all images in input_dir (e sue sottocartelle) a target_size
    e ricrea la struttura di directory corrispondente in output_dir.
    """
    # Crea la cartella di base di output
    os.makedirs(output_dir, exist_ok=True)
    
    # Cammina tutta la gerarchia di input_dir
    for root, dirs, files in os.walk(input_dir):
        # Calcola percorso relativo rispetto a input_dir
        rel_path = os.path.relpath(root, input_dir)
        # Costruisci la cartella di destinazione corrispondente
        target_dir = os.path.join(output_dir, rel_path)
        os.makedirs(target_dir, exist_ok=True)
        
        # Elabora tutti i file in questa cartella
        for filename in tqdm(files, desc=f"Processing {rel_path}"):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                input_path = os.path.join(root, filename)
                output_path = os.path.join(target_dir, filename)
                with Image.open(input_path) as img:
                    img_resized = img.resize(target_size, Image.LANCZOS)
                    img_resized.save(output_path)
    
    print(f"All images have been downsampled and saved to: {output_dir}")


# Cell 3: Example Usage
# Replace the following paths with your own directories, then uncomment to run:
input_dir = "test"
output_dir = "128x128_images"
output_dir = os.path.join(output_dir, input_dir)
downsample_images(input_dir, output_dir)


Processing .: 0it [00:00, ?it/s]
Processing C081: 100%|██████████| 327/327 [00:07<00:00, 42.14it/s]

All images have been downsampled and saved to: 128x128_images\test


In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from diffusers import UNet2DModel, DDPMScheduler, DDIMScheduler
from tqdm import tqdm

# Configuration
data_dir = "./128x128_images"  # Base directory containing class subfolders
train_dir = os.path.join(data_dir, "train")  # e.g., 128x128_images/train/C0XX
val_dir = os.path.join(data_dir, "validation")  # e.g., 128x128_images/validation/C0XX
batch_size = 32
image_size = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 50
learning_rate = 1e-4

# Data transformations
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
])

# Datasets and loaders
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Define model and scheduler
unet = UNet2DModel(
    sample_size=image_size,
    in_channels=3,
    out_channels=3,
    layers_per_block=2,
    block_out_channels=(64, 128, 256, 512),
    down_block_types=("DownBlock2D", "DownBlock2D", "DownBlock2D", "DownBlock2D"),
    up_block_types=("UpBlock2D", "UpBlock2D", "UpBlock2D", "UpBlock2D"),
).to(device)

# Choose scheduler: DDIM for sampling, but use DDPMScheduler for training noise schedule
noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
# For sampling: ddim_scheduler = DDIMScheduler.from_config(noise_scheduler.config)

optimizer = torch.optim.Adam(unet.parameters(), lr=learning_rate)

# Training loop
def train():
    unet.train()
    for epoch in range(num_epochs):
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for images, _ in loop:
            images = images.to(device)
            batch_size = images.size(0)

            # Sample noise and timesteps
            noise = torch.randn_like(images)
            timesteps = torch.randint(0, noise_scheduler.num_train_timesteps, (batch_size,), device=device)

            # Add noise to images
            noisy_images = noise_scheduler.add_noise(images, noise, timesteps)

            # Predict noise
            noise_pred = unet(noisy_images, timesteps).sample

            # Compute loss
            loss = nn.MSELoss()(noise_pred, noise)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loop.set_postfix(loss=loss.item())

        # Validation and sample generation every few epochs
        if (epoch + 1) % 10 == 0:
            sample_images(epoch + 1)

    # Save final model
    torch.save(unet.state_dict(), "ddim_unet.pth")

# Sampling for validation
def sample_images(epoch, num_samples=4):
    unet.eval()
    # ddim scheduler for sampling
    ddim_scheduler = DDIMScheduler.from_config(noise_scheduler.config)

    # Start from pure noise
    latent = torch.randn((num_samples, 3, image_size, image_size), device=device)
    for t in tqdm(list(range(ddim_scheduler.config.num_train_timesteps))[::-1], desc="Sampling"):
        # Predict noise
        with torch.no_grad():
            noise_pred = unet(latent, torch.tensor([t]*num_samples, device=device)).sample

        # Compute previous sample
        latent = ddim_scheduler.step(noise_pred, t, latent).prev_sample

    # Denormalize and save
    samples = (latent.clamp(-1, 1) + 1) / 2
    os.makedirs("samples", exist_ok=True)
    for i, img in enumerate(samples):
        transforms.ToPILImage()(img.cpu()).save(f"samples/sample_{epoch}_{i}.png")
    unet.train()


train()